In [ ]:
import keras as K
import numpy as np
import os
from keras.utils import to_categorical

In [2]:
import _pickle as pickle

In [3]:
from pathlib import Path

In [ ]:
from models import *
from datautils import *
import tensorflow as tf


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

%reload_ext autoreload
%autoreload 2

# word2vec

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

In [6]:
file = './data/corpus.csv'
with open(file, 'r') as f:
    t = f.readlines()

In [7]:
text = []
for line in t:
    text.append(line.split('\t')[0])
s = "\n".join(text)

In [8]:
s[:100]

'内测玩到现在，不管怎么样，五星\n游戏的新手指印和可玩模式有欠缺\n好玩 估计可以玩很多天不会腻~\n四月末了，是不是要改成五月份呀\n什么都好，就是电池太难耍刷到了\n具体是4月几号安卓可以下载啊？\n为什么我'

In [11]:
w2vTXT = s
with open('./data/w2vT.txt', 'w') as f:
    f.write(w2vTXT)

In [12]:
w2v = " ".join(jieba.cut(w2vTXT))

KeyboardInterrupt: 

In [ ]:
w2v[:100]

In [18]:
with open('./data/w2v.txt', 'w') as f:
    f.writelines(w2v)

In [20]:
model = Word2Vec(LineSentence('./data/w2v.txt'), size=256, window=5, min_count=5, workers=8)
model.save('./cache/w2v.model')

In [21]:
weights = model.wv.syn0
vocab = dict([(k, v.index+1) for k, v in model.wv.vocab.items()])
embed_weights = np.zeros(shape=(weights.shape[0]+2, weights.shape[1]))
embed_weights[1:weights.shape[0]+1] = weights
unk_vec = np.random.random(size=weights.shape[1]) * 0.5
embed_weights[weights.shape[0]+1] = unk_vec - unk_vec.mean()

In [44]:
len(vocab)

44409

In [43]:
pickle.dump(vocab, open('./cache/w2v.dict.pkl', 'wb'))
np.save('./cache/word_embed.npy', embed_weights)

In [40]:
model.similar_by_word('杀')

[('打死', 0.6563826203346252),
 ('弄死', 0.6158390045166016),
 ('杀掉', 0.6087832450866699),
 ('杀人', 0.6080974340438843),
 ('打爆', 0.5926108360290527),
 ('杀死', 0.5809358954429626),
 ('灭', 0.5806133151054382),
 ('碰见', 0.5758534073829651),
 ('团灭', 0.5728330016136169),
 ('灭队', 0.5724929571151733)]

# to one-hot, truncate and padding

In [6]:
sentence_length = 150
sen_seq_length = 10
word_seq_length = 15

In [7]:
with open('./cache/text.csv', 'r') as f:
    txt = f.readlines()
text = [line.strip() for line in txt]
with open('./cache/label.csv', 'r') as f:
    label = f.readlines()
labels = [int(l.strip())-1 for l in label]

In [45]:
word_embed_dict = pickle.load(open('./cache/w2v.dict.pkl', 'rb'))

In [17]:
word_unknow = len(word_embed_dict.keys()) + 1

In [36]:
word_embed_dict['我']

5

In [37]:
X = get_sequence(text[:100])
X.shape

100%|██████████| 100/100 [00:00<00:00, 49642.61it/s]


(100, 150)

In [39]:
X[0]

array([   5,  118,  245,   25,   42,    4,    1,  173,    9, 8321, 2515,
         35,    5,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], d

# dataset splits

In [4]:
with open('./cache/text.csv', 'r') as f:
    txt = f.readlines()
text = [line.strip() for line in txt]
with open('./cache/label.csv', 'r') as f:
    label = f.readlines()
labels = [int(l.strip())-1 for l in label]


sampleNB = len(labels)
shuffle_idx = np.arange(sampleNB)
np.random.shuffle(shuffle_idx)
text = np.array(text)
labels = np.array(labels)
text = text[shuffle_idx]
labels = labels[shuffle_idx]

In [5]:
train_test_split = 0.2
train_val_split = 0.1

train_test_split_idx = int(len(labels)*train_test_split)
train_text = text[train_test_split_idx:]
train_label = labels[train_test_split_idx:]
test_text = text[:train_test_split_idx]
test_label = labels[:train_test_split_idx]

train_val_split_idx = int(len(labels)*train_val_split)
val_text = train_text[:train_val_split_idx]
val_label = train_label[:train_val_split_idx]
train_text = train_text[train_val_split_idx:]
train_label = train_label[train_val_split_idx:]

In [6]:
np.save('./cache/train_label.npy', train_label)
np.save('./cache/test_label.npy', test_label)
np.save('./cache/val_label.npy', val_label)

In [7]:
np.save('./cache/train_text.npy', train_text)
np.save('./cache/val_text.npy', val_text)
np.save('./cache/test_text.npy', test_text)

# data preprocess

In [4]:
train_text = np.load('./cache/train_text.npy')
train_label = np.load('./cache/train_label.npy')

In [24]:
train_text.shape

(130687,)

In [28]:
test_text = np.load('./cache/test_text.npy')
test_label = np.load('./cache/test_label.npy')


# Classifier

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

%reload_ext autoreload
%autoreload 2

## data preparation

In [5]:
word_embed_weight=np.load('./cache/word_embed.npy')

In [8]:
val_y = np.load('./cache/val_label.npy')

In [13]:
val_X = np.load('./cache/val_X.npy')


In [14]:
val_y[:100]

array([0, 1, 3, 0, 0, 4, 4, 2, 0, 4, 3, 0, 4, 1, 0, 3, 4, 3, 0, 3, 3, 1,
       0, 0, 4, 4, 4, 4, 4, 3, 3, 4, 1, 3, 1, 2, 4, 3, 3, 4, 3, 4, 2, 4,
       4, 3, 0, 4, 2, 4, 4, 4, 2, 0, 2, 0, 0, 0, 4, 4, 2, 4, 4, 2, 0, 2,
       4, 3, 1, 4, 3, 2, 3, 2, 4, 3, 0, 4, 1, 2, 3, 3, 4, 4, 3, 3, 2, 4,
       3, 2, 4, 2, 4, 4, 3, 3, 4, 4, 1, 2])

In [12]:
val_X = word_text_preprocess(val_text)
np.save('./cache/val_X.npy', val_X)

In [9]:
val_y = to_categorical(val_y)
class_nb = val_y.shape[1]
class_nb

5

## TextCNN

In [29]:
# data
#test_X = word_text_preprocess(test_text)
train_X = word_text_preprocess(train_text)

100%|██████████| 130687/130687 [00:02<00:00, 62787.99it/s]


In [30]:
test_X = np.load('./cache/test_X.npy')
val_X = np.load('./cache/val_X.npy')
train_X = np.load('./cache/train_X.npy')

FileNotFoundError: [Errno 2] No such file or directory: './cache/train_X.npy'

In [15]:
np.save('./cache/test_X.npy', test_X)

In [17]:
batch_size=128
model_name='textcnn'
model=get_textcnn(sentence_length, word_embed_weight, class_nb)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 256)     11369216    input_1[0][0]                    
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 150, 256)     65792       embedding_1[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 150, 256)     1024        time_distributed_1[0][0]         
__________________________________________________________________________________________________
activation

In [18]:
train_batch_generator = word_text_train_batch_generator

In [19]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 38s 37ms/step - loss: 1.1899 - acc: 0.5149 - val_loss: 1.0960 - val_acc: 0.5576


In [20]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 1.0951 - acc: 0.5570 - val_loss: 1.0726 - val_acc: 0.5690


In [21]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 1.0610 - acc: 0.5705 - val_loss: 1.1017 - val_acc: 0.5621


In [22]:
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 1.0342 - acc: 0.5829 - val_loss: 1.0507 - val_acc: 0.5799
Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 1.0040 - acc: 0.5949 - val_loss: 1.0561 - val_acc: 0.5816


In [23]:
K.set_value(model.optimizer.lr, 0.0001)
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 0.9255 - acc: 0.6284 - val_loss: 1.0447 - val_acc: 0.5862
Epoch 1/1
1020/1020 [==============================] - 35s 34ms/step - loss: 0.8928 - acc: 0.6410 - val_loss: 1.0539 - val_acc: 0.5837


# HAN

In [6]:
sen_seq_length = 10
word_seq_length = 15

In [12]:
val_text = np.load('./cache/val_text.npy')
val_X = word_han_preprocess(val_text)

In [19]:
train_batch_generator = word_han_train_batch_generator
batch_size=128
model_name='han'
model=get_han(sen_seq_length, word_seq_length, word_embed_weight, class_nb)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 10, 15)            0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 256)           11665167  
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 10, 256)           295680    
_________________________________________________________________
attention_8 (Attention)      (None, 256)               266       
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

In [20]:
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 97s 95ms/step - loss: 1.1787 - acc: 0.5209 - val_loss: 1.1485 - val_acc: 0.5343
Epoch 1/1
1020/1020 [==============================] - 95s 94ms/step - loss: 1.0974 - acc: 0.5564 - val_loss: 1.1179 - val_acc: 0.5420


In [21]:
for i in range(2):
    model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 95s 93ms/step - loss: 1.0523 - acc: 0.5752 - val_loss: 1.0969 - val_acc: 0.5574
Epoch 1/1
1020/1020 [==============================] - 95s 93ms/step - loss: 1.0082 - acc: 0.5930 - val_loss: 1.0943 - val_acc: 0.5630


In [22]:
K.set_value(model.optimizer.lr, 0.0001)
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 95s 93ms/step - loss: 0.9029 - acc: 0.6376 - val_loss: 1.1130 - val_acc: 0.5639


In [23]:
model.fit_generator(train_batch_generator(train_text, train_label, batch_size=batch_size), 
                    epochs=1, steps_per_epoch=train_text.shape[0]//batch_size, validation_data=(val_X, val_y))

Epoch 1/1
1020/1020 [==============================] - 95s 93ms/step - loss: 0.8667 - acc: 0.6531 - val_loss: 1.1294 - val_acc: 0.5607
